# Random Forest

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("dataset/CL-dataset.csv",sep='\t',decimal=",",index_col=0)

In [2]:
df.head()

,Imax,E,PIL,Mt,TsM,TsW,NbxM,NbxW,MeanD,Etrim,Emonth,Eday_of_week,Ehour,class
0,12,2.807355,2.567,15.261429,8.217692,2.015660,0.076923,0.018868,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,12,7.102441,2.567,3.573791,50.033077,12.272264,0.307692,0.075472,55.500000,1.570383,1.570383,1.570383,1.570383,0
2,24,3.000000,2.567,13.797500,8.490769,2.082642,0.076923,0.018868,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,6,3.906891,2.567,4.162000,4.802308,1.177925,0.230769,0.056604,0.333333,0.000000,0.000000,0.353359,0.353359,0
4,48,6.984210,2.567,8.364100,153.770769,37.717358,0.846154,0.207547,24.454545,1.826220,2.894170,1.878620,2.382467,0


In [3]:
from sklearn.model_selection import train_test_split
label = df.pop('class')
train_set, test_set, train_label, test_label = train_test_split(df, label, stratify =label, test_size=0.30)

In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


## Grid search

In [5]:


param_dist = [{"max_depth": [2,3,12,None],
              "max_features": [5],# Al massimo 6 non di più
              "min_samples_split": [10,50],
              "min_samples_leaf": [10,50],
              "bootstrap": [True, False],
              "criterion": ["entropy", "gini"],
              "class_weight":['balanced', None, {0: 0.3, 1: 0.7}]}]

clf = RandomForestClassifier(n_estimators=30)
grid_search = GridSearchCV(clf, param_dist,  
                            n_jobs=3, 
                            scoring='accuracy')

grid_search.fit(train_set, train_label)

print("Complete Grid search:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

print("Best parameters:",grid_search.best_params_)
print("Best Score: ",grid_search.best_score_)


s_leaf': 10, 'min_samples_split': 10}
0.598 (+/-0.075) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 50}
0.598 (+/-0.052) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 50, 'min_samples_split': 10}
0.618 (+/-0.080) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 50, 'min_samples_split': 50}
0.726 (+/-0.023) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 10}
0.731 (+/-0.027) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 50}
0.732 (+/-0.023) for {'bootstrap': True, 'class_weight': {0: 0

## Modello Finale

In [6]:
rf = RandomForestClassifier(n_estimators=30, 
                             criterion='entropy',
                             class_weight=None,
                             max_features=5,
                             max_depth=12, 
                             min_samples_split=50,
                             min_samples_leaf=10,
                             bootstrap=True) 
rf = rf.fit(train_set, train_label)

In [7]:
#prediction on the test test
test_pred_rf = rf.predict(test_set)

In [8]:
#compute the performance of the model
print(classification_report(test_label, 
                            test_pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97       507
           1       0.92      0.97      0.94       507
           2       0.97      0.97      0.97       507

    accuracy                           0.96      1521
   macro avg       0.96      0.96      0.96      1521
weighted avg       0.96      0.96      0.96      1521

